In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

import lsst.sims.maf.db as db
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.plots as plots
import numpy as np

In [2]:
import lsst.sims.maf
lsst.sims.maf.__version__

'2.8.0.sims-22-gab2f692'

In [3]:
# opsdb = db.OpsimDatabase(runName+'.db')
runName = 'baseline2018a'
# opsdb = db.OpsimDatabase('../../../mothra_2048/data/mothra_2048.db')
opsdb = db.OpsimDatabase('baseline2018a.db')
outDir = 'stackers_test'
sqlconstraint = 'night<730'
resultsDb = db.ResultsDb(outDir=outDir)

In [4]:
slicer = slicers.HealpixSlicer(nside=64)
stackerList = [stackers.NormAirmassStacker(degrees=True),stackers.HourAngleStacker()]

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [5]:
metric = metrics.MedianMetric('normairmass')

In [6]:
bundle = metricBundles.MetricBundle(metric, slicer, sqlconstraint, 
                                    runName=runName,
                                    stackerList=stackerList)
bgroup = metricBundles.MetricBundleGroup({'normairmass': bundle}, 
                                         opsdb, outDir=outDir, 
                                         resultsDb=resultsDb)

In [7]:
bgroup.runAll()

Querying database SummaryAllProps with constraint night<730 for columns ['airmass', 'fieldDec', 'fieldRA', 'observationStartLST']
Found 452269 visits
Running:  ['normairmass']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [8]:
print(bgroup.simData.dtype.names)

('airmass', 'fieldDec', 'fieldRA', 'observationStartLST', 'normairmass', 'HA')


In [9]:
bgroup.simData

array([ ( 1.00478859, -24.889283,  305.088793,  305.02475404,  1.00040268, -0.00426926),
       ( 2.01707179,  29.966322,  302.89208 ,  305.38824654,  1.00209729,  0.1664111 ),
       ( 1.18659642, -62.802603,  305.4822  ,  306.1194096 ,  1.00011976,  0.04248064),
       ...,
       ( 1.10892466,  -4.728603,   76.678001,   78.67264158,  1.00076543,  0.13297604),
       ( 1.05076528, -12.40057 ,   78.116992,   78.83976457,  1.00021715,  0.04818484),
       ( 1.00345109, -26.053426,   76.714953,   79.14058594,  1.00076756,  0.16170886)],
      dtype=[('airmass', '<f8'), ('fieldDec', '<f8'), ('fieldRA', '<f8'), ('observationStartLST', '<f8'), ('normairmass', '<f8'), ('HA', '<f8')])

Use this field (These values are just taken from)

RA = 306.24892991

DEC = -62.802603

HA = 0.04248064

Opsim Airmass = 1.18659642

In [10]:
telescope_lat = -30.2444

In [11]:
# convert HA in hours to radians
ha_rads = (np.pi*0.04248064)/12.0

In [12]:
dec = np.radians(-62.802603)
min_z_possible = np.abs(dec - np.radians(telescope_lat))
min_airmass_possible = 1./np.cos(min_z_possible)

In [13]:
min_airmass_possible

1.1864574844694964

In [14]:
cos_z = (np.sin(np.radians(telescope_lat))*np.sin(dec)
         +np.cos(np.radians(telescope_lat))*np.cos(dec)*np.cos(ha_rads))
calculated_airmass = 1/cos_z
calculated_airmass

1.186491858343973

In [15]:
calculated_airmass/min_airmass_possible

1.00002897185523

In [16]:
# convert HA in hours to radians
1.18659642 - calculated_airmass

0.00010456165602712453

In [17]:
#percent change
((1.18659642 - calculated_airmass)/1.18659642)*100

0.0088118971425115641

In [18]:
# opsdb.fetchMetricData(['airmass','fieldRA','fieldDec','observationStartMJD'])